In [ ]:
import croissant as cro
import croissant.jax as crojax
from functools import partial
from healpy import get_nside, projview
import jax
import jax.numpy as jnp
import lunarsky
import matplotlib.pyplot as plt
import s2fft

In [ ]:
# simulation parameters
world = "moon"
lmax = 179  # maximal harmonic band limit given sampling of beam, inclusive
freq = jnp.arange(1, 51)  # 1-50 MHz
time = lunarsky.Time("2025-12-01 09:00:00")  # time at the beginning of the simulation
loc = lunarsky.MoonLocation(lon=0, lat=-22.5)  # location of telescope
topo = lunarsky.LunarTopo(obstime=time, location=loc) # coordinate frame of telescope
# 240 bins in a sidereal day on the moon
ntimes = 240
dt = cro.constants.sidereal_day[world] / ntimes
phases = crojax.simulator.rot_alm_z(lmax, ntimes, dt, world=world)

# get the euler angles and wigner d functions for the coordinate transforms
eul_topo, dl_topo = crojax.rotations.generate_euler_dl(lmax, topo, "mcmf")  # beam transform, from topocentric to mcmf
eul_gal, dl_gal = crojax.rotations.generate_euler_dl(lmax, "galactic", "mcmf")  # sky transform, from galactic to mcmf

topo2mcmf = partial(s2fft.utils.rotation.rotate_flms, L=lmax+1, rotation=eul_topo, dl_array=dl_topo)
gal2mcmf = partial(s2fft.utils.rotation.rotate_flms, L=lmax+1, rotation=eul_gal, dl_array=dl_gal)

In [ ]:
beam = jnp.load("beam.npy")
plt.figure()
plt.imshow(beam[30], aspect="auto")
plt.xlabel("$\\phi$ [deg]")
plt.ylabel("$\\theta$ [deg]")
plt.colorbar()
plt.show()

In [ ]:
# define the map2alm transform for the beam
beam2alm = partial(s2fft.forward_jax, L=lmax+1, spin=0, nside=None, sampling="mwss", reality=True)
# use vmap t vectorize frequency axis
beam_alm = jax.vmap(beam2alm)(beam)

# normalization for visibilities
norm = crojax.alm.total_power(beam_alm, lmax)

In [ ]:
# sky
sky_map = jnp.load("ulsa.npy")
ix = -6
projview(m=sky_map[ix], title=f"ULSA sky at {freq[ix]} MHz")
plt.show()

In [ ]:
# define the map2alm transform for the sky
nside = get_nside(sky_map[0])
sky2alm = partial(s2fft.forward_jax, L=lmax+1, spin=0, nside=nside, sampling="healpix", reality=True)
sky_alm = jax.vmap(sky2alm)(sky_map)

In [ ]:
# coordinate transform
print(crojax.alm.is_real(beam_alm))
print(crojax.alm.is_real(sky_alm))
beam_alm = jax.vmap(topo2mcmf)(beam_alm)
sky_alm = jax.vmap(gal2mcmf)(sky_alm)
print(crojax.alm.is_real(beam_alm))
print(crojax.alm.is_real(sky_alm))

In [ ]:
# beam and sky as seen by simulator after transformed back to healpix
beam_reconstructed = s2fft.inverse_jax(beam_alm[ix], L=lmax+1, spin=0, nside=nside, sampling="healpix", reality=True)
sky_reconstructed = s2fft.inverse_jax(sky_alm[ix], L=lmax+1, spin=0, nside=nside, sampling="healpix", reality=True).real

projview(m=beam_reconstructed.real, title=f"Reconstructed beam at {freq[ix]} MHz")
projview(m=sky_reconstructed, title=f"Reconstructed sky at {freq[ix]} MHz")
plt.show()

In [ ]:
vis = crojax.simulator.convolve(beam_alm, sky_alm, phases).real / norm

In [ ]:
plt.figure()
plt.plot(freq, jnp.mean(sky_map, axis=-1))
plt.plot(freq, jnp.mean(sky_map, axis=-1)[0] * (freq/freq.min())**(-2.5))
plt.yscale("log")
plt.show()

In [ ]:
plt.figure()
plt.imshow(jnp.log(vis), aspect="auto")
plt.colorbar()
plt.show()

In [ ]:
plt.figure()
plt.plot(freq, vis[150])
plt.yscale("log")
plt.show()